## 릿지 회귀

- 선형 모델에 L2 제약조건을 추가한 Ridge 클래스
- 주요 생성 파라미터: alpha

[L2 제약조건]
- 모든 특성에 대한 가중치의 값을 0 주변으로 위치하도록 제어하는 제약조건(0이 되지는 않음)
- 데이터의 컬럼들을 하나의 컬럼에 가중치가 극대화되는 것을 막으면서 모든 컬럼들을 최소한으로 사용하여 회귀

In [11]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score

In [6]:
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['target'] = boston.target

In [7]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


### 규제 없이 머신 러닝 성능 파악

In [25]:
model = LinearRegression()
scores = cross_val_score(model, boston.data, boston.target, cv=5,
                        scoring = 'neg_mean_squared_error')
print('NMSE: {}'.format(scores))
print('MSE: {}'.format(-1 * scores))
print('MSE Mean: {:.3f}'.format(np.mean(-1 * scores)))

NMSE: [-12.46030057 -26.04862111 -33.07413798 -80.76237112 -33.31360656]
MSE: [12.46030057 26.04862111 33.07413798 80.76237112 33.31360656]
MSE Mean: 37.132


교차 검증 결과 평균 MSE의 값이 37.132로 나왔다.

### 규제를 가하여 머신 러닝 성능 파악(릿지 회귀)

In [18]:
from sklearn.linear_model import Ridge

model = Ridge(alpha = 10)
scores = cross_val_score(model, boston.data, boston.target, cv=5,
                        scoring = 'neg_mean_squared_error')
print('NMSE: {}'.format(scores))
print('MSE: {}'.format(-1 * scores))
print('MSE Mean: {:.3f}'.format(np.mean(-1 * scores)))

NMSE: [-11.4217962  -24.29446706 -28.14355069 -74.59916618 -28.51714218]
MSE: [11.4217962  24.29446706 28.14355069 74.59916618 28.51714218]
MSE Mean: 33.395


교차 검증 결과 평균 MSE의 값이 33.395로 나왔다. 규제가 없는 것보다 좋은 예측 성능을 보여준다.

이번에는 릿지의 alpha 값을 바꾸면서 변화를 파악했다.

In [21]:
alpha = [0, .1, 1, 10, 100]

for a in alpha:
    model = Ridge(alpha = a)
    
    scores = cross_val_score(model, boston.data, boston.target, cv = 5,
                            scoring = 'neg_mean_squared_error')
    print('alpha 값이 {1}일 때, MSE Mean: {:.3f}'.format(a, np.mean(-1 * scores)))

alpha 값이 0일 때, MSE Mean: 37.132
alpha 값이 0.1일 때, MSE Mean: 36.707
alpha 값이 1일 때, MSE Mean: 35.267
alpha 값이 10일 때, MSE Mean: 33.395
alpha 값이 100일 때, MSE Mean: 29.906


alpha 값이 커질수록 회귀 계수 값을 작게 만드는 것을 볼 수 있다.

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=.2)

alpha = [0, .1, 1, 10, 100]

for a in alpha:
    model = Ridge(alpha = a)
    model.fit(X_train, y_train)
    print(model.coef_)

[-9.10922498e-02  3.95639653e-02 -2.12734706e-02  3.27109209e+00
 -2.07404196e+01  3.57832532e+00  5.86983480e-03 -1.61077020e+00
  3.93213279e-01 -1.45846590e-02 -1.09755810e+00  9.65732659e-03
 -5.17373988e-01]
[-9.00231089e-02  3.98647953e-02 -2.68748348e-02  3.24542544e+00
 -1.91485555e+01  3.59001169e+00  4.49910511e-03 -1.58395593e+00
  3.91195705e-01 -1.47856637e-02 -1.08031578e+00  9.75552085e-03
 -5.19631560e-01]
[-8.49025411e-02  4.14261867e-02 -5.42525787e-02  3.07488762e+00
 -1.13217006e+01  3.63482018e+00 -2.07935701e-03 -1.45234889e+00
  3.82076565e-01 -1.58105968e-02 -9.96674553e-01  1.02396984e-02
 -5.31814393e-01]
[-0.08099298  0.04460015 -0.0843765   2.25829476 -2.21383317  3.45682226
 -0.00700764 -1.30057419  0.38412978 -0.01756888 -0.91665889  0.01079708
 -0.5647637 ]
[-0.08560172  0.05102423 -0.08959335  0.68192712 -0.23047521  2.04635637
  0.00738724 -1.18310748  0.43299357 -0.02003122 -0.94266899  0.0104626
 -0.67310067]


alpha 값이 증가하면서 회귀 계수가 지속적으로 작아지고 있음을 알 수 있으며, 릿지 회귀의 경우에는 회귀 계수를 0 으로 만들지는 않는다는 특징을 발견할 수 있다.

---

## 라쏘 회귀

- 선형 모델에 L1 제약조건으 추가한 Lasso 클래스
- 주요 생성 파라미터: alpha

[L1 제약조건]
- 중요하지 않은 항목(특성, 칼럼)의 coefficient를 0으로 바꿈
- 대다수의 특성에 대해서는 coefficient를 0으로 바꾸고, 유의미한 종속관계가 보이는 것만 남기는 역할

### 규제 없이 머신 러닝 성능 파악

In [42]:
model = LinearRegression()
scores = cross_val_score(model, boston.data, boston.target, cv=5,
                        scoring = 'neg_mean_squared_error')
print('NMSE: {}'.format(scores))
print('MSE: {}'.format(-1 * scores))
print('MSE Mean: {:.3f}'.format(np.mean(-1 * scores)))

NMSE: [-12.46030057 -26.04862111 -33.07413798 -80.76237112 -33.31360656]
MSE: [12.46030057 26.04862111 33.07413798 80.76237112 33.31360656]
MSE Mean: 37.132


교차 검증 결과 평균 MSE의 값이 37.132로 나왔다.

### 규제를 가하여 머신 러닝 성능 파악(라쏘 회귀)

In [49]:
from sklearn.linear_model import Lasso

model = Lasso(alpha = 0.1)
scores = cross_val_score(model, boston.data, boston.target, cv=5,
                        scoring = 'neg_mean_squared_error')
print('NMSE: {}'.format(scores))
print('MSE: {}'.format(-1 * scores))
print('MSE Mean: {:.3f}'.format(np.mean(-1 * scores)))

NMSE: [-11.22162227 -24.34502474 -29.42629614 -79.36695204 -29.79226639]
MSE: [11.22162227 24.34502474 29.42629614 79.36695204 29.79226639]
MSE Mean: 34.830


교차 검증 결과 평균 MSE의 값이 34.830로 나왔다. 규제가 없는 것보다 좋은 예측 성능을 보여준다.

이번에는 라쏘의 alpha 값을 바꾸면서 변화를 파악했다.

In [53]:
alpha = [.0001, .001, .01, .1]

for a in alpha:
    model = Lasso(alpha = a)
    
    scores = cross_val_score(model, boston.data, boston.target, cv = 5,
                            scoring = 'neg_mean_squared_error')
    print('alpha 값이 {:}일 때, MSE Mean: {:.3f}'.format(a, np.mean(-1 * scores)))

alpha 값이 0.0001일 때, MSE Mean: 37.121
alpha 값이 0.001일 때, MSE Mean: 37.026
alpha 값이 0.01일 때, MSE Mean: 36.201
alpha 값이 0.1일 때, MSE Mean: 34.830


alpha 값이 커질수록 회귀 계수 값을 작게 만드는 것을 볼 수 있다.

---

## 엘라스틱넷 회귀

- L2 규제와 L1 규제를 결합한 회귀
- 라쏘 회귀가 서로 상관관계가 높은 피처들의 경우에 이들 중에서 중요 피처만을 선택하고 다른 피처들은 모두 회귀 계수를 0으로 만드는 성향이 강하며, 특히 이러한 특징으로 인해 alpha 값에 따라 회귀 계수의 값이 급격히 변하는 것을 완화함
- 주요 생성 파라미터: alpha와 l1_ratio

### 규제 없이 머신 러닝 성능 파악

In [42]:
model = LinearRegression()
scores = cross_val_score(model, boston.data, boston.target, cv=5,
                        scoring = 'neg_mean_squared_error')
print('NMSE: {}'.format(scores))
print('MSE: {}'.format(-1 * scores))
print('MSE Mean: {:.3f}'.format(np.mean(-1 * scores)))

NMSE: [-12.46030057 -26.04862111 -33.07413798 -80.76237112 -33.31360656]
MSE: [12.46030057 26.04862111 33.07413798 80.76237112 33.31360656]
MSE Mean: 37.132


### 규제를 가하여 머신 러닝 성능 파악(엘라스틱넷 회귀)

In [56]:
from sklearn.linear_model import ElasticNet

model = ElasticNet(alpha = 0.1, l1_ratio=0.5)
scores = cross_val_score(model, boston.data, boston.target, cv = 5,
                        scoring = 'neg_mean_squared_error')
print('NMSE: {}'.format(scores))
print('MSE: {}'.format(-1 * scores))
print('MSE Mean: {:.3f}'.format(np.mean(-1 * scores)))

NMSE: [-11.67670794 -25.05332614 -30.30410068 -68.92807929 -26.64921508]
MSE: [11.67670794 25.05332614 30.30410068 68.92807929 26.64921508]
MSE Mean: 32.522


교차 검증 결과 평균 MSE의 값이 32.522로 나왔다. 규제가 없는 것보다 좋은 예측 성능을 보여준다.

이번에는 엘라스틱넷 회귀의 alpha 값을 바꾸면서 변화를 파악했다.

In [61]:
alpha = [ .001, .01, .1, 1]
l1_ratio = [.1, .3, .5]

for a in alpha:
    for l in l1_ratio:
        model = ElasticNet(alpha = a, l1_ratio = l)
        scores = cross_val_score(model, boston.data, boston.target, cv = 5,
                             scoring = 'neg_mean_squared_error')
        print('alpha 값이 {:}이고 l1_ratio 값이 {:}일 때, MSE Mean: {:.3f}'.format(a, l, np.mean(-1 * scores)))

alpha 값이 0.001이고 l1_ratio 값이 0.1일 때, MSE Mean: 36.002
alpha 값이 0.001이고 l1_ratio 값이 0.3일 때, MSE Mean: 36.158
alpha 값이 0.001이고 l1_ratio 값이 0.5일 때, MSE Mean: 36.345
alpha 값이 0.01이고 l1_ratio 값이 0.1일 때, MSE Mean: 34.360
alpha 값이 0.01이고 l1_ratio 값이 0.3일 때, MSE Mean: 34.526
alpha 값이 0.01이고 l1_ratio 값이 0.5일 때, MSE Mean: 34.723
alpha 값이 0.1이고 l1_ratio 값이 0.1일 때, MSE Mean: 31.351
alpha 값이 0.1이고 l1_ratio 값이 0.3일 때, MSE Mean: 31.873
alpha 값이 0.1이고 l1_ratio 값이 0.5일 때, MSE Mean: 32.522
alpha 값이 1이고 l1_ratio 값이 0.1일 때, MSE Mean: 30.861
alpha 값이 1이고 l1_ratio 값이 0.3일 때, MSE Mean: 31.359
alpha 값이 1이고 l1_ratio 값이 0.5일 때, MSE Mean: 32.054


alpha 값이 1이고 l1_ratio 값이 0.1일 때, MSE의 평균값이 가장 작다.